In [1]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

postsDf = pd.read_csv("information_for_mongo.csv")

In [2]:
#time conversion and edit formats
from datetime import datetime
import time

postsDf.reset_index(drop=True)
postsDf = postsDf.fillna(value="None")
postsDf.drop(postsDf.columns[postsDf.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

for index, row in postsDf.iterrows():
    if row["is_event"] == True:
      row_dict = {}
      row_dict = eval(row["event_details"])
      try:
        conStart = datetime.fromisoformat(row_dict["start_time"])
        unixStart = time.mktime(conStart.timetuple())
        try:
          conEnd = datetime.fromisoformat(row_dict["end_time"])
          unixEnd = time.mktime(conEnd.timetuple())
        except:
          conEnd = None
        row_dict["start_time"]=unixStart
        row_dict["end_time"]=unixEnd
      except:
        postsDf.at[index, "is_event"] = False
        err = f'Start failed {row_dict["start_time"]}'
        print(index, err)
      postsDf.at[index, "event_details"] = row_dict
      print(unixStart,unixEnd)

1728021600.0 1728194400.0
1728021600.0 1728194400.0
1721350800.0 1721356200.0
44 Start failed null
1721350800.0 1721356200.0
1721322000.0 1721332800.0
52 Start failed There is no specific time mentioned
1721322000.0 1721332800.0
1720460700.0 1721332800.0
1720641600.0 1720648800.0
58 Start failed None
1720641600.0 1720648800.0
1720576800.0 1720584000.0
1720490400.0 1720497600.0
1720288800.0 1720306800.0
1720717200.0 1720728000.0
1719360000.0 1719367200.0
74 Start failed E5 Bridge, 2024-06-27T18:00:00+00:00
1719360000.0 1719367200.0
75 Start failed null
1719360000.0 1719367200.0
76 Start failed None
1719360000.0 1719367200.0
1721347200.0 1721358000.0
79 Start failed value not found
1721347200.0 1721358000.0
1721433600.0 1721358000.0
1721354700.0 1721358000.0
86 Start failed Not Found
1721354700.0 1721358000.0
87 Start failed null
1721354700.0 1721358000.0
1718236800.0 1718247600.0
1717804800.0 1717808400.0
97 Start failed null
1717804800.0 1717808400.0
1717027200.0 1717808400.0
103 Start

In [ ]:
#resetting db if nexessary
load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

# collection.delete_many({})

In [4]:
from typing import List
from together import Together
import json
import time

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

for index, row in postsDf.iterrows():
    if row["is_event"] == True:
      row_dict = {}
      for column in postsDf.columns.tolist():
        if column in ['account','date','caption','accessibility_caption','url','likes','display_photo']:
          row_dict[column] = row[column]
      row_dict["embedded"] = generate_embedding([row["processed_json"]], embedding_model_string)  
      row_dict["event_details"] = row["event_details"]
      result = collection.insert_one(row_dict)
      print(f"Inserted document ID: {result.inserted_id}")
      time.sleep(1)



        

Inserted document ID: 66ce4fa6d5089f799a66be68
Inserted document ID: 66ce4fa7d5089f799a66be69
Inserted document ID: 66ce4fa9d5089f799a66be6a
Inserted document ID: 66ce4faad5089f799a66be6b
Inserted document ID: 66ce4fabd5089f799a66be6c
Inserted document ID: 66ce4facd5089f799a66be6d
Inserted document ID: 66ce4faed5089f799a66be6e
Inserted document ID: 66ce4fafd5089f799a66be6f
Inserted document ID: 66ce4fb1d5089f799a66be70
Inserted document ID: 66ce4fb2d5089f799a66be71
Inserted document ID: 66ce4fb4d5089f799a66be72
Inserted document ID: 66ce4fb5d5089f799a66be73
Inserted document ID: 66ce4fb6d5089f799a66be74
Inserted document ID: 66ce4fb7d5089f799a66be75
Inserted document ID: 66ce4fb9d5089f799a66be76
Inserted document ID: 66ce4fbad5089f799a66be77
Inserted document ID: 66ce4fbbd5089f799a66be78
Inserted document ID: 66ce4fbdd5089f799a66be79
Inserted document ID: 66ce4fbed5089f799a66be7a
Inserted document ID: 66ce4fbfd5089f799a66be7b
Inserted document ID: 66ce4fc1d5089f799a66be7c
Inserted docu

In [9]:

def return_events(prompt):
    query = prompt
    query_emb = generate_embedding([query], embedding_model_string)[0]

    results = collection.aggregate([
    {
        "$vectorSearch": {
        "queryVector": query_emb,
        "path": "embedded",
        "numCandidates": 100, # this should be 10-20x the limit
        "limit": 2, # the number of documents to return in the results
        "index": "vector_index", # the index name you used in Step 4.
        }
    }
    ])
    response = ""
    for i in results:
        response += (i['caption']+ "\n"+ i['url'] + '\n\n')
    return response

prompt = "Cheese"

print(return_events(prompt))


🔥✨ Join us for our Bonfire End of Term event in collaboration with our friends at CSC, WiCS, and Tech+ ✨🔥

📅 Tuesday, July 23rd
⏰ 6:00 - 9:00 PM
📍Columbia Lake, Fire Pit 1

Please note this event is drop in! No need to stay for the whole time, come and go as you please, open to UWaterloo students😄

Get ready for a fun evening with s'mores, hot dogs (vegan option available), drinks, freezies, and other snacks🍫😋

Don't miss out! See you there 🌐

Sign up using the link in bio🔗
C9i2rTsgxql

Want to gain hands-on experience with product development and improvement? Join us at our upcoming PRODUCT DEVELOPMENT NIGHT!!✨

💼 This event will feature a short lecture about the essentials of product development followed by a collaborative case study facilitated by our very own UW PM execs.

🗓️ Thursday, July 25th
⏰ 6:30 - 8 PM
📍E7 2nd Floor Ideas Clinic (2409)
🥤Free Chungchun for everyone who signs up beforehand😋

Link in bio to sign up :)
C9vfxOhg0hw


